# VideoRobot — One-Click Colab Runner
این نوت‌بوک ریپو را کلون می‌کند، FFmpeg و وابستگی‌ها را نصب می‌کند، بک‌اند را بالا می‌آورد، یک مانیفست نمونه می‌سازد، رندر می‌زند و MP4 نهایی را برمی‌گرداند.

In [ ]:
# 1) Clone repo (fresh)
import shutil, pathlib
REPO_URL = "https://github.com/englishpodcasteasy-glitch/videorobot.git"
WORK = pathlib.Path("/content/videorobot")
shutil.rmtree(WORK, ignore_errors=True)
!git clone {REPO_URL} {WORK}
!cd {WORK} && git rev-parse --short HEAD

In [ ]:
# 2) Install FFmpeg and Python deps
!bash /content/videorobot/scripts/install_ffmpeg_colab.sh || (apt-get update && apt-get install -y ffmpeg)
!pip install -r /content/videorobot/backend/requirements.txt

In [ ]:
# 3) Start backend on port 8000 and health-check
import os, time, requests, subprocess, sys
os.environ["BACKEND_PORT"] = os.environ.get("BACKEND_PORT", "8000")
backend = subprocess.Popen([sys.executable, "backend/main.py"], cwd="/content/videorobot")
base = f"http://127.0.0.1:{os.environ['BACKEND_PORT']}"
for i in range(120):  # Colab گاهی کند است؛ 2 دقیقه صبر کن
    try:
        r = requests.get(base + "/healthz", timeout=2)
        if r.ok:
            print("Health OK:", r.text)
            break
    except Exception:
        pass
    time.sleep(1)
else:
    raise SystemExit("Backend did not become healthy in time")

In [ ]:
# 4) Create a simple manifest
manifest = {
  "seed": 7,
  "video": {"width": 720, "height": 1280, "fps": 30, "bg_color": "#101318"},
  "tracks": [
    {"type": "text", "content": "سلام دنیا", "start": 0.2, "duration": 3, "x": 40, "y": 80, "size": 72, "color": "#FFFFFF"}
  ]
}
import json, pathlib
path = pathlib.Path("/content/manifest.json")
path.write_text(json.dumps(manifest, ensure_ascii=False))
print("Manifest:", path)

In [ ]:
# 5) Submit render, poll, and download result
import requests, time, os, json, urllib.request
base = f"http://127.0.0.1:{os.environ['BACKEND_PORT']}"
resp = requests.post(base + "/render", json=json.loads(open("/content/manifest.json", "r", encoding="utf-8").read()), timeout=60)
print("POST /render:", resp.status_code, resp.text)
resp.raise_for_status()
job_id = resp.json().get("job_id")
assert job_id, "No job_id returned"
for i in range(600):
    r = requests.get(base + f"/progress/{job_id}", timeout=10)
    if r.ok:
        js = r.json(); print(f"tick {i}:", js.get("state"), js.get("pct"))
        if js.get("state") == "success":
            break
        if js.get("state") == "error":
            raise SystemExit(js)
    time.sleep(1)
out = f"/content/{job_id}.mp4"
urllib.request.urlretrieve(base + f"/download?jobId={job_id}", out)
print("Downloaded:", out)
print("If not found here, also check /content/outputs/<job_id>/final.mp4")